# Detectability Map
@version: 2
@author: Max Felius
@date: 17/05/2021

This is a new version of the detectability map.

In [1]:
#imports
import numpy as np
import pandas as pd
import geopandas as gpd
import sys, os, time
import matplotlib.pyplot as plt
from tqdm import tqdm
from shapely.geometry import Point
from scipy import spatial

# import personal functions 
sys.path.extend(os.path.join(os.getcwd(),'sinkhole_functions'))
from sinkhole_functions.functions import * 
from sinkhole_functions.geometric_models import *
from sinkhole_functions.detectability_map import *

# Loading data

In [2]:
folder = ''
filename = 't88_points_Franciscanerstraat_r100.csv'

dataset_filename = os.path.join(folder,filename)

data = pd.read_csv(dataset_filename,index_col=0)
geo_data = gpd.GeoDataFrame(data.drop(['geometry'],axis=1),crs={'init':'epsg:4326'},geometry=[Point(xy) for xy in zip(data.pnt_lon, data.pnt_lat)])

#other crs
geo_data = geo_data.to_crs('epsg:28992')

rdx = geo_data.geometry.x.values
rdy = geo_data.geometry.y.values

rd_data = np.concatenate((rdx.reshape(len(rdx),1),rdy.reshape(len(rdy),1)),axis=1)
tree = spatial.cKDTree(rd_data)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [3]:
#make a grid of the data
def make_grid_list(data,posting=1):
    '''
    Returns the x and y coordinates of the posting together with a list of the indeces of the coordinates
    
    :type data: geopandas
    :rtype: 
    '''
    #number of points
    n = len(data)
    
    rdx = data.geometry.x.values
    rdy = data.geometry.y.values
    
    #get the maximum and minimum values for the coordinates
    xmin = np.min(rdx)
    xmax = np.max(rdx)
    ymin = np.min(rdy)
    ymax = np.max(rdy)
    
    #make the grid
    x_range = np.arange(xmin,xmax,posting)
    y_range = np.arange(ymin,ymax,posting)
    nx, ny = np.meshgrid(x_range,y_range)
    
    index_list = np.arange(0,len(nx.ravel()),1)
    
    return nx, ny, index_list

#implement function
posting = 1
nx, ny, grid_list = make_grid_list(geo_data,posting)

In [6]:
def evaluate_posting(data,tree,nx,ny,R):
    '''
    Function evaluating each posting on the possibility of the kinematic model
    '''
    nx_u = nx.ravel()
    ny_u = ny.ravel()
    
    #create index list
    index_list = np.arange(0,len(nx_u),1)
    
    #get the data
    rdx = data.geometry.x.values
    rdy = data.geometry.y.values
    
    data_out = np.concatenate((nx_u.reshape((len(nx_u),1)),ny_u.reshape((len(ny_u),1)),np.zeros((len(nx_u),1))),axis=1)
    
    data_return = pd.DataFrame(data_out,columns=['grid_rdx','grid_rdy','Sum'])
    
    t = np.array([0, 11, 22, 33, 44, 55, 66, 77, 88, 99, 110, 121, 132, 165])
    b = t.reshape((len(t),1))
    
    for ii,iR in enumerate(R):
        temp_data = pd.DataFrame(np.zeros((len(nx_u),1)),columns=[f'{iR}'])

        for idx in tqdm(index_list,f'{ii+1}/{len(R)}'):
            x0 = nx_u[idx]
            y0 = ny_u[idx]
            
            subset = tree.query_ball_point(([x0,y0]),r=iR)
            
            rdx_subset = rdx[subset]
            rdy_subset = rdy[subset]
            
            r = np.sqrt((rdx_subset-x0)**2 + (rdy_subset-y0)**2)

            if len(r)<=1:
                np.delete(index_list,np.where(index_list==idx))
            else:
                #defining the jacobian matrix for nonlinear least squares
                a = gaussian(iR,r).reshape((len(r),1))
                z = a @ b.T
                z = z.ravel()

                A = np.array([z])
                cond_number = np.linalg.cond(A)
                
                if cond_number < 10:
                    temp_data.iloc[idx] = 1
                    data_return['Sum'].iloc[idx] = data_return['Sum'].iloc[idx] + 1
                else:
                    np.delete(index_list,np.where(index_list==idx))

        data_return = data_return.join(temp_data)
    return data_return

R_vector = np.arange(100,0,-1)

data_return = evaluate_posting(geo_data,tree,nx,ny,R_vector)

100/100: 100%|██████████| 98141/98141 [01:03<00:00, 1545.76it/s]


In [10]:
posting = 1

In [12]:
def make_map(rdx,rdy,posting):
    '''
    something
    '''
    df = pd.DataFrame(np.zeros((len(rdx),1)),columns=['geometry (rd)'])
    half = posting/2
    
    for i in tqdm(range(len(rdx)),'Making the map'):
        x = rdx[i]
        y = rdy[i]
        
        lbrd_x = x-half
        lbrd_y = y-half
        rbrd_x = x+half
        rbrd_y = y-half
        rtrd_x = x+half 
        rtrd_y = y+half
        ltrd_x = x-half
        ltrd_y = y+half
        
        df['geometry (rd)'].iloc[i] = f'POLYGON (({lbrd_x} {lbrd_y},{rbrd_x} {rbrd_y},{rtrd_x} {rtrd_y},{ltrd_x} {ltrd_y}))'
    
    return df

df_out = make_map(nx.ravel(),ny.ravel(),posting)

new_data = data_return.join(df_out)

Making the map: 100%|██████████| 98141/98141 [03:41<00:00, 443.43it/s]


In [13]:
new_data.to_csv('Detectability_Map_Posting1m_Franciscanerstraat.csv')